In [1]:
import concurrent.futures
import ansys.additive.additive as pyadditive
from ansys.additive.machine import AdditiveMachine
from ansys.additive.single_bead import (
    BeadType,
    SingleBeadInput,
)
import matplotlib.pyplot as plt
import numpy as np

In [2]:
additive = pyadditive.launch_additive(ip='localhost', port=50052)

INFO:ansys.additive.additive:Connected to localhost:50052


In [3]:
# material_list = additive.get_materials_list()
# print(material_list)

material = additive.get_material('17-4PH')
machine = AdditiveMachine();
machine.scan_speed = 1.25
machine.laser_power = 350
input = SingleBeadInput(
    machine=machine,
    material=material,
    id="single-bead",
    bead_length=0.001,
    bead_type=BeadType.BEAD_ON_POWDER,
)
result = additive.simulate(input)    

Single Bead simulation: 100%|██████████| 100/100 [00:15<00:00,  6.67it/s]


In [4]:
print(result)

# power 50 - 700
# velocity .350 - 2.500
# pv = []
# for p in [50, 350, 700]:
#   for v in [.35, 1.25, 2.5]:
#     pv.append((p,v))
# pv = [(350,1.25)]

SingleBeadSummary
input: SingleBeadInput
id: single-bead
bead_length: 0.001
bead_type: BeadType.BEAD_ON_POWDER

machine: AdditiveMachine
laser_power: 350
scan_speed: 1.25
heater_temperature: 80
layer_thickness: 5e-05
beam_diameter: 0.0001
starting_layer_angle: 57
layer_rotation_angle: 67
hatch_spacing: 0.0001
slicing_stripe_width: 0.01

material: AdditiveMaterial
absorptivity_maximum: 0.8
absorptivity_minimum: 0.2
absorptivity_powder_coefficient_a: 0.104021901
absorptivity_powder_coefficient_b: 0.444062208
absorptivity_solid_coefficient_a: 0.104021901
absorptivity_solid_coefficient_b: 0.444062208
anisotropic_strain_coefficient_parallel: 1.5
anisotropic_strain_coefficient_perpendicular: 0.5
anisotropic_strain_coefficient_z: 1.0
elastic_modulus: 175000000000.0
hardening_factor: 0.019943
liquidus_temperature: 1450.0
material_yield_strength: 821000000.0
name: 17-4PH
nucleation_constant_bulk: 28444.3999
nucleation_constant_interface: 2.37783356
penetration_depth_maximum: 0.00041
penetration

In [ ]:
# results = []
# with concurrent.futures.ThreadPoolExecutor(4) as executor:
#     futures = []
#     for x in pv:
#         machine = AdditiveMachine.default_machine()
#         machine.laser_power=x[0]
#         machine.scan_speed=x[1] 
#         futures.append(executor.submit(additive.simulate_single_bead, machine=machine, material=material, bead_length=0.001, bead_type=BeadType.BEAD_ON_POWDER))
#     for future in concurrent.futures.as_completed(futures):
#         results.append(future.result())



In [ ]:
print(result)
# results.sort(key=lambda r : (r.machine.laser_power, r.machine.scan_speed))
# # for r in range(len(results)):
# #     print("power " + str(results[r].machine.laser_power) + ", scan speed " + str(results[r].machine.scan_speed))
# # plt.style.use('_mpl-gallery')
# # #fig, ax = plt.subplots(figsize=(5, 2.7), layout='constrained')
# nrows = 3
# ncols = 3
# fig, axs = plt.subplots(nrows, ncols, figsize=(15, 15), layout='constrained')
# for r in range(nrows):
#     for c in range(ncols):
#         i = r * nrows + c
#         axs[r][c].plot(results[i].melt_pool.laser_x_mm, results[i].melt_pool.width_mm, label='width')
#         axs[r][c].plot(results[i].melt_pool.laser_x_mm, results[i].melt_pool.reference_width_mm, label='ref width')
#         axs[r][c].plot(results[i].melt_pool.laser_x_mm, results[i].melt_pool.depth_mm, label='depth')
#         axs[r][c].plot(results[i].melt_pool.laser_x_mm, results[i].melt_pool.reference_depth_mm, label='ref depth')
#         axs[r][c].plot(results[i].melt_pool.laser_x_mm, results[i].melt_pool.length_mm, label='length')
#         axs[r][c].set_ylim(0, 1.1)
#         axs[r][c].legend()
#         axs[r][c].set_xlabel(f'Bead Length (mm)')  # Add an x-label to the axes.
#         axs[r][c].set_ylabel(f'(mm)')  # Add a y-label to the axes.
#         title = "Power " + str(results[i].machine.laser_power) + "w, Scan Speed " + str(results[i].machine.scan_speed) + "m/s"
#         axs[r][c].set_title(title)  # Add a title to the axes.

In [ ]:
powers = []
velocities = []
dw = []
lw = []

for r in results:
    m = r.melt_pool
    m.ave_length = np.average(m.length_mm)
    m.ave_width = np.average(m.width_mm)
    m.ave_depth = np.average(m.depth_mm)
    m.ave_reference_width = np.average(m.reference_width_mm)
    m.ave_reference_depth = np.average(m.reference_depth_mm)
    powers.append(r.machine.laser_power)
    velocities.append(r.machine.scan_speed)
    dw.append(m.ave_depth / m.ave_width if m.ave_width else 0)
    lw.append(m.ave_length / m.ave_width if m.ave_width else 0)

In [ ]:
from matplotlib.colors import LinearSegmentedColormap as colorMap

dwMin=0.37
dwMax=0.6
contour_gradient = []  # Someday switch this to numpy
marker_colors = []
fig, ax = plt.subplots(1, 2, figsize=(20,10))
for i in range(len(dw)):
    if dwMin < dw[i] < dwMax:
        contour_gradient.append(0)
        marker_colors.append('blue')
    else:
        contour_gradient.append(abs(((dwMax + dwMin) / 2) - dw[i]))
        marker_colors.append('black')
    txt = "{}".format(round(dw[i], 2))
    ax[0].annotate(str(txt), (velocities[i], powers[i] + 25), verticalalignment='top', horizontalalignment='center')

colMap = colorMap.from_list("", ["green", "yellow", "red", "red"])
ax[0].tricontourf(velocities, powers, contour_gradient, cmap=colMap, levels=255)
ax[0].scatter(velocities, powers, c=marker_colors, marker='d', s=100, facecolor='none')
ax[0].use_sticky_edges = False
ax[0].margins(0.075)
ax[0].set_title('Melt Pool Depth/Width')
ax[0].set_xlabel(f'Laser Scan Speed (m/s)')
ax[0].set_ylabel(f'Laser Power (w)')

lwMax = 3.5
contour_gradient = []
marker_colors = []
colMap = ['#ff0000ff', '#ff6600dd', '#ffff0022', '#00000000']
for i in range(len(lw)):
    if lw[i] > lwMax:
        contour_gradient.append(0)
        marker_colors.append('blue')
    else:
        contour_gradient.append(abs(lwMax - lw[i]))
        marker_colors.append('black')
    txt = "{}".format(round(lw[i], 2))
    ax[1].annotate(str(txt), (velocities[i], powers[i] + 25), verticalalignment='top', horizontalalignment='center')

# cmap = colorMap.from_list("", colMap)
cmap = colorMap.from_list("", ["green", "yellow", "red", "red"])
ax[1].tricontourf(velocities, powers, contour_gradient, cmap=cmap, levels=255)
ax[1].scatter(velocities, powers, c=marker_colors, marker='d', s=100, facecolor='none')
ax[1].use_sticky_edges = False
ax[1].margins(0.075)
ax[1].set_title('Melt Pool Length/Width')
ax[1].set_xlabel(f'Laser Scan Speed (m/s)')
ax[1].set_ylabel(f'Laser Power (w)')

 